In [1]:
import config
import tweepy
import pandas as pd

In [96]:
df = pd.read_csv("UserData.csv")

In [3]:
auth = tweepy.OAuthHandler(config.key,config.scret_key)
auth.set_access_token(config.access_token,config.access_token_secret)
api = tweepy.API(auth)

In [119]:
mentions = api.mentions_timeline()

for mention in mentions:
    splitt = mention.text.split(" ")
    username = mention.user.screen_name
    tweetID = mention.id
    # try:
    #
    #     api.destroy_status(tweetID)
    #     print("Booooom")
    # except:
    #     print("Boom")
    #     continue
    try:
        pin = splitt[2]
        age = int(splitt[3].split("+")[0])
        reply = "@" + username + " You have been successfuly subscribed. I will be reminding you when vaccine slot is available at your pincode " + pin
        if df["Username"].any()==username :
            reply = "@" + username + " " + "You are already subscribed"
        else:
            df = df.append({ "Username": username,"TweetID": tweetID, "Pin": pin, "Age": age},ignore_index=True)
        api.update_status(reply,tweetID)
        print(pin,age,username,tweetID)
    except:
        print("Wrong tweet")
        reply = "@" + username + " " + 'There seems to be an error. Please make sure your tweet is in the format ". @ getmyslot subscribe <Pin Code> <Age>" An example of valid query will be following'
        file = open("ss.jpeg" , "rb")
        r1 = api.media_upload(filename="ss.jpeg",file=file)
        api.update_status(reply,tweetID,media_ids=[r1.media_id_string])
        continue

In [121]:
df.to_csv("UserData.csv",index=False)
